# Causal Imitation Learning Examples - Behavioral Cloning

In [1]:
import numpy as np

N = 100000
rng = np.random.default_rng(0)

# utility
def logistic(size=None):
    return rng.logistic(loc=0.0, scale=1.0, size=size)

### 9.18: BC in MAB

In [2]:
p = 0.9 # P(U=1) = 0.9
theta = 2.0 # reward scale (any positive number would work)

In [3]:
# generate expert data
U = rng.binomial(1, p, size=N)
X_expert = U.copy()
Y_expert = X_expert.copy()
R_expert = theta * Y_expert

p_hat = X_expert.mean()
expert_value = R_expert.mean()

In [4]:
X_bc = rng.binomial(1, p_hat, size=N)
Y_bc = X_bc.copy()
R_bc = theta * Y_bc

bc_value = R_bc.mean()

In [5]:
print(f'Ground-truth p = P(U=1) = {p:.3f}')
print(f'Reward: R(Y) = theta * Y with theta = {theta:.3f}')
print()
print('From expert rollouts:')
print(f'  Empirical P(X=1) (i.e., cloned by BC)   p_hat = {p_hat:.5f}')
print(f'  Expert value (empirical)                 = {expert_value:.5f}')
print(f'  Expert value (analytic)                  = {(theta * p):.5f}')
print()
print('Behavioral Cloning (BC):')
print(f'  BC policy uses pi_BC(X=1) = p_hat        = {p_hat:.5f}')
print(f'  BC value (empirical rollout)             = {bc_value:.5f}')
print(f'  BC value (analytic, using p_hat)         = {(theta * p_hat):.5f}')
print()
print('Takeaway: since Y := X deterministically, E_pi[R] = theta * pi(X=1).')
print('Here pi_BC(X=1) = P_expert(X=1) = p, so BC exactly matches the expert.')
print()
# should be close to 0
print(f'  |expert_emp - expert_analytic|  = {abs(expert_value - theta * p):.5e}')
print(f'  |bc_emp - expert_analytic|      = {abs(bc_value - theta * p):.5e}')


Ground-truth p = P(U=1) = 0.900
Reward: R(Y) = theta * Y with theta = 2.000

From expert rollouts:
  Empirical P(X=1) (i.e., cloned by BC)   p_hat = 0.90095
  Expert value (empirical)                 = 1.80190
  Expert value (analytic)                  = 1.80000

Behavioral Cloning (BC):
  BC policy uses pi_BC(X=1) = p_hat        = 0.90095
  BC value (empirical rollout)             = 1.80478
  BC value (analytic, using p_hat)         = 1.80190

Takeaway: since Y := X deterministically, E_pi[R] = theta * pi(X=1).
Here pi_BC(X=1) = P_expert(X=1) = p, so BC exactly matches the expert.

  |expert_emp - expert_analytic|  = 1.90000e-03
  |bc_emp - expert_analytic|      = 4.78000e-03


### 9.19: BC in DTR

In [6]:
def simulate_expert(N):
    # follow SCM spec from Eq. 8.2 w/ alpha1 := 0, alpha2 := 0
    U = rng.binomial(1, 0.5, size=N)
    U3 = logistic(size=N)
    S1 = (U3 > 0).astype(int)

    U1 = logistic(size=N)
    X1 = (3*S1+ U1 > 0).astype(int)

    U4 = logistic(size=N)
    S2 = (0.1 + 0.1*S1 + 0.1*X1 + U4 > 0).astype(int)

    U2 = logistic(size=N)
    X2 = (3*S2 + U2 > 0).astype(int)

    Y = (3*U - 3*S1 - 3*X1 - 3*S1*X1 + 3*X2 - 3*S2*X2 + 3*X1*X2 > 0).astype(int)

    return dict(S1=S1, X1=X1, S2=S2, X2=X2, Y=Y)

In [7]:
def fit_bc_tables(trajs):
    S1, X1, S2, X2 = trajs['S1'], trajs['X1'], trajs['S2'], trajs['X2']

    p_x1 = np.zeros(2)
    for s in (0, 1):
        mask = (S1 == s)

        # smoothing to avoid 0/0
        p_x1[s] = (X1[mask].sum() + 1e-6) / (mask.sum() + 2e-6)

    p_x2 = np.zeros((2, 2, 2))
    for s in (0, 1):
        for x1 in (0, 1):
            for s2 in (0, 1):
                mask = (S1 == s) & (X1 == x1) & (S2 == s2)
                p_x2[s, x1, s2] = (X2[mask].sum() + 1e-6) / (mask.sum() + 2e-6)

    return p_x1, p_x2

In [8]:
def rollout_bc(N, p_x1, p_x2):
    U = rng.binomial(1, 0.5, size=N)
    U3 = logistic(size=N)
    S1 = (U3 > 0).astype(int)

    probs1 = p_x1[S1]
    X1 = (rng.random(N) < probs1).astype(int)

    U4 = logistic(size=N)
    S2 = (0.1 + 0.1*S1 + 0.1*X1 + U4 > 0).astype(int)

    probs2 = p_x2[S1, X1, S2]
    X2 = (rng.random(N) < probs2).astype(int)

    Y = (3*U - 3*S1 - 3*X1 - 3*S1*X1 + 3*X2 - 3*S2*X2 + 3*X1*X2 > 0).astype(int)

    return dict(S1=S1, X1=X1, S2=S2, X2=X2, Y=Y)

In [9]:
N_eval = N

expert_trajs = simulate_expert(N)
p_x1, p_x2 = fit_bc_tables(expert_trajs)

expert_eval = simulate_expert(N_eval)
expert_value = expert_eval['Y'].mean()

bc_eval = rollout_bc(N_eval, p_x1, p_x2)
bc_value = bc_eval['Y'].mean()

In [10]:
print("SCM (Eq. 8.2 with α1=α2=0):")
print("  S1 <- I{ U3 > 0 }")
print("  X1 <- I{ 3*S1 + α1*U + U1 > 0 }  (α1=0 here)")
print("  S2 <- I{ 0.1 + 0.1*S1 + 0.1*X1 + U4 > 0 }")
print("  X2 <- I{ 3*S2 + α2*U + U2 > 0 }  (α2=0 here)")
print("  Y  <- I{ 3U - 3S1 - 3X1 - 3*S1*X1 + 3*X2 - 3*S2*X2 + 3*X1*X2 > 0 }")
print("Reward R = Y.\n")
print()
print(f'Estimated BC tables (few entries): P(X1=1|S1): {p_x1}')
print(f'P(X2=1|S1,X1,S2) for S1=1, X1=0, S2=: {p_x2[1,0,:]}') # slice of P(X2=1|S1,X1,S2)
print()
print(f'Expert value (mean Y) over {N_eval:,} eval rollouts: {expert_value:.5f}')
print(f'BC value     (mean Y) over {N_eval:,} eval rollouts: {bc_value:.5f}')
print(f'Absolute difference |BC - Expert|: {abs(bc_value - expert_value):.3e}')

SCM (Eq. 8.2 with α1=α2=0):
  S1 <- I{ U3 > 0 }
  X1 <- I{ 3*S1 + α1*U + U1 > 0 }  (α1=0 here)
  S2 <- I{ 0.1 + 0.1*S1 + 0.1*X1 + U4 > 0 }
  X2 <- I{ 3*S2 + α2*U + U2 > 0 }  (α2=0 here)
  Y  <- I{ 3U - 3S1 - 3X1 - 3*S1*X1 + 3*X2 - 3*S2*X2 + 3*X1*X2 > 0 }
Reward R = Y.


Estimated BC tables (few entries): P(X1=1|S1): [0.50030082 0.95220999]
P(X2=1|S1,X1,S2) for S1=1, X1=0, S2=: [0.51545455 0.96219136]

Expert value (mean Y) over 100,000 eval rollouts: 0.28020
BC value     (mean Y) over 100,000 eval rollouts: 0.27776
Absolute difference |BC - Expert|: 2.440e-03


### 9.20: BC fails without NUC

In [11]:
theta = 2.0
N_train = N
N_eval = N

def xor(a, b):
    return (a ^ b).astype(int)

In [12]:
# generate expert data
U_train = rng.binomial(1, 0.5, size=N_train)
X_train = 1 - U_train
Y_train = xor(X_train, U_train)
R_train = theta * Y_train
p_hat = X_train.mean()

In [13]:
# evaluate expert
U_eval = rng.binomial(1, 0.5, size=N_eval)
X_expert = 1 - U_eval
Y_expert = xor(X_expert, U_eval)
R_expert = theta * Y_expert
expert_value = R_expert.mean()

In [14]:
# evaluate BC
X_bc = rng.binomial(1, p_hat, size=N_eval)
Y_bc = xor(X_bc, U_eval)
R_bc = theta * Y_bc
bc_value = R_bc.mean()

In [15]:
print(f"Reward: R(Y) = theta * Y with theta = {theta:.3f}")
print()
print("Expert (has access to U): X := NOT U; Y := XOR(X, U) = 1")
print(f"  Expert value (empirical) = {expert_value:.5f}")
print(f"  Expert value (analytic)  = {theta:.5f}")
print()
print("Behavioral Cloning (imitator does NOT see U):")
print(f"  Cloned marginal P(X=1) from expert data: p_hat = {p_hat:.5f}")
print("  Deploy X ~ Bernoulli(p_hat) independent of U ~ Bernoulli(0.5)")
print(f"  BC value (empirical)      = {bc_value:.5f}")
print(f"  BC value (analytic)       = {(theta * 0.5):.5f}")
print()
print("Takeaway: BC breaks the expert's X–U coupling that made Y=1 always;")
print("with unobserved confounding, BC only achieves E[Y]=0.5 (half the expert).")
print()
print(f"  |expert_emp - expert_analytic| = {abs(expert_value - theta):.5e}")
print(f"  |bc_emp - bc_analytic|         = {abs(bc_value - (theta * 0.5)):.5e}")

Reward: R(Y) = theta * Y with theta = 2.000

Expert (has access to U): X := NOT U; Y := XOR(X, U) = 1
  Expert value (empirical) = 2.00000
  Expert value (analytic)  = 2.00000

Behavioral Cloning (imitator does NOT see U):
  Cloned marginal P(X=1) from expert data: p_hat = 0.50091
  Deploy X ~ Bernoulli(p_hat) independent of U ~ Bernoulli(0.5)
  BC value (empirical)      = 1.00050
  BC value (analytic)       = 1.00000

Takeaway: BC breaks the expert's X–U coupling that made Y=1 always;
with unobserved confounding, BC only achieves E[Y]=0.5 (half the expert).

  |expert_emp - expert_analytic| = 0.00000e+00
  |bc_emp - bc_analytic|         = 5.00000e-04


### 9.21: Imitation Backdoor does not imply Sequential Backdoor

In [16]:
def simulate_expert(N):
    # use graph from example
    Z = rng.binomial(1, 0.5, size=N)

    U1 = logistic(size=N)
    X1 = (U1 > 0).astype(int)

    U2 = logistic(size=N)
    X2 = (3*X1 + Z + U2 > 0).astype(int)

    UY = logistic(size=N)
    Y = (Z + 3*X2 + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, X2=X2, Y=Y)

In [17]:
def fit_imitation_tables(trajs):
    Z, X1, X2 = trajs['Z'], trajs['X1'], trajs['X2']
    p_x1 = X1.mean()

    p_x2 = np.zeros(2)
    for z in (0, 1):
        mask = (Z == z)
        p_x2[z] = (X2[mask].sum() + 1e-6) / (mask.sum() + 2e-6) # smoothing to avoid 0/0

    return p_x1, p_x2

In [18]:
def rollout_imitation(N, p_x1, p_x2):
    Z = rng.binomial(1, 0.5, size=N)

    X1 = (rng.random(N) < p_x1).astype(int)

    probs2 = p_x2[Z]
    X2 = (rng.random(N) < probs2).astype(int)

    UY = logistic(size=N)
    Y = (Z + 3*X2 + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, X2=X2, Y=Y)

In [19]:
N_train = N
N_eval = N

expert_trajs = simulate_expert(N_train)
p_x1_hat, p_x2_hat = fit_imitation_tables(expert_trajs)

expert_eval = simulate_expert(N_eval)
expert_value = expert_eval['Y'].mean()

imitation_eval = rollout_imitation(N_eval, p_x1_hat, p_x2_hat)
imitation_value = imitation_eval['Y'].mean()

In [20]:
print("SCM (9.21-style):")
print("  Z  ~ Bern(0.5)")
print("  X1 <- I{ U1 > 0 }")
print("  X2 <- I{ 3*X1 + 1*Z + U2 > 0 }")
print("  Y  <- I{ 1*Z + 3*X2 + UY > 0 }")
print()
print("Policy spaces:")
print("  Imitation policy:  pi1(X1)=P(X1),  pi2(X2|Z)=P(X2|Z)")
print("    (cuts X1 -> X2, so X1 is not an ancestor of Y in the manipulated graph;")
print("     Z blocks the backdoor X2 <- Z -> Y.)")
print()

print(f"Estimated tables:  P(X1=1)={p_x1_hat:.3f},   P(X2=1|Z=0)={p_x2_hat[0]:.3f},  P(X2=1|Z=1)={p_x2_hat[1]:.3f}")
print()
print(f'Expert value (mean Y) over {N_eval:,} eval rollouts:     {expert_value:.5f}')
print(f'Imitation value (mean Y) over {N_eval:,} eval rollouts:  {imitation_value:.5f}')
print(f'Absolute difference |Imitation - Expert|:                {abs(imitation_value - expert_value):.3e}')

SCM (9.21-style):
  Z  ~ Bern(0.5)
  X1 <- I{ U1 > 0 }
  X2 <- I{ 3*X1 + 1*Z + U2 > 0 }
  Y  <- I{ 1*Z + 3*X2 + UY > 0 }

Policy spaces:
  Imitation policy:  pi1(X1)=P(X1),  pi2(X2|Z)=P(X2|Z)
    (cuts X1 -> X2, so X1 is not an ancestor of Y in the manipulated graph;
     Z blocks the backdoor X2 <- Z -> Y.)

Estimated tables:  P(X1=1)=0.502,   P(X2=1|Z=0)=0.730,  P(X2=1|Z=1)=0.854

Expert value (mean Y) over 100,000 eval rollouts:     0.88661
Imitation value (mean Y) over 100,000 eval rollouts:  0.88765
Absolute difference |Imitation - Expert|:                1.040e-03


### 9.22: Sequential Backdoor does not imply Imitation Backdoor

In [21]:
def simulate_expert(N):
    Z = rng.binomial(1, 0.5, size=N)

    U1 = logistic(size=N)
    X1 = (1.5*Z + U1 > 0).astype(int)

    UW = logistic(size=N)
    W = (1.5*X1 + 1.5*Z + UW > 0).astype(int)

    U2 = logistic(size=N)
    X2 = (2*X1 + 2*Z + U2 > 0).astype(int)

    UY = logistic(size=N)
    Y = (1.8*X2 + 1.2*Z + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, W=W, X2=X2, Y=Y)

In [22]:
def fit_imitation_tables(trajs):
    Z, X1, W, X2, Y = (trajs[k] for k in ('Z', 'X1', 'W', 'X2', 'Y'))
    eps = 1e-6

    pz = np.array([(Z == 0).mean(), (Z == 1).mean()]) # P(Z)

    # pi1
    p_x1_z = np.zeros(2)
    for z in (0, 1):
        mask = (Z == z)
        p_x1_z[z] = (X1[mask].sum() + eps) / (mask.sum() + 2*eps)

    # pi2
    p_x2_w = np.zeros(2)
    for w in (0, 1):
        mask = (W == w)
        p_x2_w[w] = (X2[mask].sum() + eps) / (mask.sum() + 2*eps)

    # P(W=1 | Z, X1)
    p_w_zx1 = np.zeros((2, 2))
    for z in (0, 1):
        for x1 in (0, 1):
            mask = (Z == z) & (X1 == x1)
            p_w_zx1[z, x1] = (W[mask].sum() + eps) / (mask.sum() + 2*eps)

    # P(Y=1 | Z, X2)
    p_y_zx2 = np.zeros((2, 2))
    for z in (0, 1):
        for x2 in (0, 1):
            mask = (Z == z) & (X2 == x2)
            p_y_zx2[z, x2] = (Y[mask].sum() + eps) / (mask.sum() + 2*eps)

    return dict(pz=pz, p_x1_z=p_x1_z, p_x2_w=p_x2_w, p_w_zx1=p_w_zx1, p_y_zx2=p_y_zx2)

In [23]:
def rollout_imitation(N, tables):
    # true rollout under non-admissible imitation policy
    p_x1_z, p_x2_w = tables['p_x1_z'], tables['p_x2_w']

    Z = rng.binomial(1, 0.5, size=N)

    probs1 = p_x1_z[Z]
    X1 = (rng.random(N) < probs1).astype(int)

    UW = logistic(size=N)
    W = (1.5*X1 + 1.5*Z + UW > 0).astype(int)

    probs_x2 = p_x2_w[W]
    X2 = (rng.random(N) < probs_x2).astype(int)

    UY = logistic(size=N)
    Y = (1.8*X2 + 1.2*Z + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, W=W, X2=X2, Y=Y)

In [24]:
def sequential_backdoor_value(tables):
    pz, p_x1_z = tables['pz'], tables['p_x1_z']
    p_x2_w, p_w_zx1, p_y_zx2 = tables['p_x2_w'], tables['p_w_zx1'], tables['p_y_zx2']

    V = 0.0
    for z in (0, 1):
        for x1 in (0, 1):
            for w in (0, 1):
                # mixture over X2 under policy pi2(.|w)
                Ey_given_z = (1 - p_x2_w[w]) * p_y_zx2[z, 0] + p_x2_w[w] * p_y_zx2[z, 1]
                V += pz[z] * \
                    ((1 - p_x1_z[z]) if x1 == 0 else p_x1_z[z]) * \
                    ((1 - p_w_zx1[z, x1]) if w == 0 else p_w_zx1[z, x1]) * \
                    Ey_given_z

    return V

In [25]:
N_train = N
N_eval = N

expert_trajs = simulate_expert(N_train)
tables = fit_imitation_tables(expert_trajs)

expert_eval = simulate_expert(N_eval)
expert_value = expert_eval['Y'].mean()

imitation_eval = rollout_imitation(N_eval, tables)
imitation_value = imitation_eval['Y'].mean()

sb_value = sequential_backdoor_value(tables)

In [26]:
print("SCM (9.22-style):")
print("  Z ~ Bern(0.5)")
print("  X1 <- I{ 1.5*Z + U1 > 0 }")
print("  W  <- I{ 1.5*X1 + 1.5*Z + UW > 0 }   (collider: X1 -> W <- Z)")
print("  X2 <- I{ 2.0*X1 + 2.0*Z + U2 > 0 }   (expert uses X1,Z)")
print("  Y  <- I{ 1.8*X2 + 1.2*Z + UY > 0 }")
print("\nPolicy spaces:")
print("  Imitation policy:  pi1(X1|Z)=P(X1|Z),   pi2(X2|W)=P(X2|W)  <-- uses collider only (fails imitation backdoor)")
print("  Sequential backdoor: evaluate V(pi) using full history (Z, X1, W) via G-computation.\n")

print(f"Expert value (mean Y) over {N_eval:,} eval rollouts:                 {expert_value:.5f}")
print(f"Imitation policy TRUE value (rollout) over {N_eval:,} eval rollouts:  {imitation_value:.5f}")
print(f"Gap |Imitation - Expert|:                                            {abs(imitation_value - expert_value):.3e}\n")

print(f"Sequential-backdoor estimate V_sb(pi) from logged data:              {sb_value:.5f}")
print(f"|V_sb(pi) - TRUE imitation value|:                                   {abs(sb_value - imitation_value):.3e}\n")

SCM (9.22-style):
  Z ~ Bern(0.5)
  X1 <- I{ 1.5*Z + U1 > 0 }
  W  <- I{ 1.5*X1 + 1.5*Z + UW > 0 }   (collider: X1 -> W <- Z)
  X2 <- I{ 2.0*X1 + 2.0*Z + U2 > 0 }   (expert uses X1,Z)
  Y  <- I{ 1.8*X2 + 1.2*Z + UY > 0 }

Policy spaces:
  Imitation policy:  pi1(X1|Z)=P(X1|Z),   pi2(X2|W)=P(X2|W)  <-- uses collider only (fails imitation backdoor)
  Sequential backdoor: evaluate V(pi) using full history (Z, X1, W) via G-computation.

Expert value (mean Y) over 100,000 eval rollouts:                 0.84728
Imitation policy TRUE value (rollout) over 100,000 eval rollouts:  0.85673
Gap |Imitation - Expert|:                                            9.450e-03

Sequential-backdoor estimate V_sb(pi) from logged data:              0.85569
|V_sb(pi) - TRUE imitation value|:                                   1.042e-03



### 9.23: Using imitation-admissible subspace for BC

In [27]:
def simulate_expert(N):
    # same as 9.22
    Z = rng.binomial(1, 0.5, size=N)

    U1 = logistic(size=N)
    X1 = (1.5*Z + U1 > 0).astype(int)

    UW = logistic(size=N)
    W = (1.5*X1 + 1.5*Z + UW > 0).astype(int)

    U2 = logistic(size=N)
    X2 = (2*X1 + 2*Z + U2 > 0).astype(int)

    UY = logistic(size=N)
    Y = (1.8*X2 + 1.2*Z + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, W=W, X2=X2, Y=Y)

In [28]:
def fit_subspace_tables(trajs):
    # admissible
    Z, X1, X2 = trajs['Z'], trajs['X1'], trajs['X2']
    eps = 1e-6

    p_x1 = (X1.sum() + eps) / (len(X1) + 2*eps)

    p_x2_z = np.zeros(2)
    for z in (0, 1):
        mask = (Z == z)
        p_x2_z[z] = (X2[mask].sum() + eps) / (mask.sum() + 2*eps)

    return dict(p_x1=p_x1, p_x2_z=p_x2_z)

In [29]:
def rollout_subspace(N, tables):
    p_x1, p_x2_z = tables['p_x1'], tables['p_x2_z']

    Z = rng.binomial(1, 0.5, size=N)

    X1 = (rng.random(N) < p_x1).astype(int)

    UW = logistic(size=N)
    W = (1.5*X1 + 1.5*Z + UW > 0).astype(int)

    probs_x2 = p_x2_z[Z]
    X2 = (rng.random(N) < probs_x2).astype(int)

    UY = logistic(size=N)
    Y = (1.8*X2 + 1.2*Z + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, W=W, X2=X2, Y=Y)

In [30]:
def fit_bad_tables(trajs):
    # non-admissible
    Z, X1, W, X2 = (trajs[k] for k in ('Z', 'X1', 'W', 'X2'))
    eps = 1e-6

    p_x1_z = np.zeros(2)
    for z in (0, 1):
        mask = (Z == z)
        p_x1_z[z] = (X1[mask].sum() + eps) / (mask.sum() + 2*eps)

    p_x2_w = np.zeros(2)
    for w in (0, 1):
        mask = (W == w)
        p_x2_w[w] = (X2[mask].sum() + eps) / (mask.sum() + 2*eps)

    return dict(p_x1_z=p_x1_z, p_x2_w=p_x2_w)

In [31]:
def rollout_bad(N, tables):
    p_x1_z, p_x2_w = tables['p_x1_z'], tables['p_x2_w']

    Z = rng.binomial(1, 0.5, size=N)

    probs1 = p_x1_z[Z]
    X1 = (rng.random(N) < probs1).astype(int)

    UW = logistic(size=N)
    W = (1.5*X1 + 1.5*Z + UW > 0).astype(int)

    probs_x2 = p_x2_w[W]
    X2 = (rng.random(N) < probs_x2).astype(int)

    UY = logistic(size=N)
    Y = (1.8*X2 + 1.2*Z + UY > 0).astype(int)

    return dict(Z=Z, X1=X1, W=W, X2=X2, Y=Y)

In [32]:
N_train = N
N_eval = N

expert_trajs = simulate_expert(N_train)
expert_evals = simulate_expert(N_eval)
expert_value = expert_evals['Y'].mean()

sub_tables = fit_subspace_tables(expert_trajs)
sub_eval = rollout_subspace(N_eval, sub_tables)
sub_value = sub_eval['Y'].mean()

bad_tables = fit_bad_tables(expert_trajs)
bad_eval = rollout_bad(N_eval, bad_tables)
bad_value = bad_eval['Y'].mean()

In [33]:
print("SCM (9.23-style, same family as 9.21–9.22):")
print("  Z  ~ Bern(0.5)")
print("  X1 <- I{ 1.5*Z + U1 > 0 }")
print("  W  <- I{ 1.5*X1 + 1.5*Z + UW > 0 }   (collider: X1 -> W <- Z)")
print("  X2 <- I{ 2.0*X1 + 2.0*Z + U2 > 0 }   (expert uses X1,Z)")
print("  Y  <- I{ 1.8*X2 + 1.2*Z + UY > 0 }")
print("\nPolicies:")
print("  • Imitation-admissible subspace:  pi1(X1)=P(X1),  pi2(X2|Z)=P(X2|Z)")
print("    (cuts X1→X2; Z blocks the backdoor X2←Z→Y)  → should match expert.")
print("  • Contrast (non-admissible):       pi1(X1|Z)=P(X1|Z),  pi2(X2|W)=P(X2|W)")
print("    (uses collider W at stage 2)     → typically ≠ expert.\n")

print(f"Expert value (mean Y) over {N_eval:,} eval rollouts:            {expert_value:.5f}")
print(f"Imitation-admissible subspace value over {N_eval:,}:             {sub_value:.5f}")
print(f" |Subspace − Expert|:                                           {abs(sub_value - expert_value):.3e}\n")

print(f"Non-admissible (collider-only at stage 2) value over {N_eval:,}: {bad_value:.5f}")
print(f" |Non-admissible − Expert|:                                     {abs(bad_value - expert_value):.3e}\n")

SCM (9.23-style, same family as 9.21–9.22):
  Z  ~ Bern(0.5)
  X1 <- I{ 1.5*Z + U1 > 0 }
  W  <- I{ 1.5*X1 + 1.5*Z + UW > 0 }   (collider: X1 -> W <- Z)
  X2 <- I{ 2.0*X1 + 2.0*Z + U2 > 0 }   (expert uses X1,Z)
  Y  <- I{ 1.8*X2 + 1.2*Z + UY > 0 }

Policies:
  • Imitation-admissible subspace:  pi1(X1)=P(X1),  pi2(X2|Z)=P(X2|Z)
    (cuts X1→X2; Z blocks the backdoor X2←Z→Y)  → should match expert.
  • Contrast (non-admissible):       pi1(X1|Z)=P(X1|Z),  pi2(X2|W)=P(X2|W)
    (uses collider W at stage 2)     → typically ≠ expert.

Expert value (mean Y) over 100,000 eval rollouts:            0.84584
Imitation-admissible subspace value over 100,000:             0.84515
 |Subspace − Expert|:                                           6.900e-04

Non-admissible (collider-only at stage 2) value over 100,000: 0.85646
 |Non-admissible − Expert|:                                     1.062e-02

